In [1]:
from pathlib import Path
import os
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
import pandas as pd

radius = 150 

root_dir = Path(os.path.abspath('')).parents[1]
exp_dir = os.path.join(root_dir, "soma_detection")
data_dir = os.path.join(exp_dir, "data")
volumes_dir = os.path.join(data_dir, "volumes", str(radius))
fig_dir = os.path.join(data_dir, "figures")

brains = [1]

cap = 180

d = []
for brain in brains:
    positive_count = 0
    negative_count = 0
    brain_name = f"brain{brain}"
    neigh_dir = os.path.join(fig_dir, brain_name, "neighbors")
    brain_volumes_dir = os.path.join(volumes_dir, brain_name)
    
    negative_filenames = os.listdir(os.path.join(neigh_dir, "0"))
    positive_filenames = os.listdir(os.path.join(neigh_dir, "1"))
    
    filenames = np.concatenate((negative_filenames, positive_filenames))
    
    r = np.random.default_rng(1234)
    r.shuffle(filenames)
    
    for filename in filenames:
        if negative_count == cap:
            break

        neigh_id = filename.split(".")[0]
            
        neighimg = np.load(os.path.join(brain_volumes_dir, f"{neigh_id}.npy"))
        proj = np.max(neighimg, 2)
            
        plt.imshow(proj)
        plt.show()
        print(f"{negative_count}/{cap} negatives")
        i = input("Is there a soma in this image? [y]es, [n]o, [m]aybe:\n")
        if i=="y":
            label = 1
            positive_count += 1
        elif i=="m":
            label = -1
        elif i=="n":
            label = 0
            negative_count += 1
        
        clear_output(wait=True)
        d.append({"id": neigh_id, "label":label})

    df = pd.DataFrame(d)
    df.to_csv(os.path.join(brain_volumes_dir, "neigh_manual_labels.csv"))
    print("Classification saved!")

Classification saved!


In [10]:
negatives = df[df["label"] == 0].to_numpy()[:,0]

_negatives = np.array([filename.split(".")[0] for filename in negative_filenames])

TN = 0
for negative in negatives:
    if negative in _negatives:
        TN += 1
TN/len(negatives)

0.9111111111111111